# Pre-Equilibrium Maximazation of Yield #

In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../../")
import copy

In [2]:
from steric_free_simulator import ReactionNetwork, VectorizedRxnNet, VecSim, Optimizer, EquilibriumSolver

EnergyExplorer Module is not available. Check Rosetta installation. <ipykernel.iostream.OutStream object at 0x7f590404e908>


We'll start with the AP2 complex that we've worked with before. Pairwise $\Delta Gs$ were derived from the PDB structures via Rossetta

In [3]:
base_input = '../../input_files/trimer.pwr'
rn = ReactionNetwork(base_input, one_step=True)
print(rn.rxn_coupling)
rn.resolve_tree()
print(rn.rxn_coupling)

['default_assoc', 1.0]
['A']
100.0
['B']
100.0
['C']
100.0
Parsing rule...
['A(a)+B(a)<->A(a!1).B(a!1)', 'G=-20']
['A', 'B', '']
['A(a)+B(a)', 'A(a!1).B(a!1)']
GGGGGGGGGgg
Parsing rule...
['A(b)+C(b)<->A(b!1).C(b!1)', 'G=-20']
['A', 'C', '']
['A(b)+C(b)', 'A(b!1).C(b!1)']
GGGGGGGGGgg
Parsing rule...
['B(b)+C(a)<->B(b!1).C(a!1)', 'G=-20']
['B', 'C', '']
['B(b)+C(a)', 'B(b!1).C(a!1)']
GGGGGGGGGgg
False
Node-1 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x7f586d70b160>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1})
Node-2 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x7f586d70b160>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1})
-----
{'A'}
{'A'}
set()
Steric hindrance detected
Node-1 :  (0, {'struct': <networkx.classes.graph.Graph object at 0x7f586d70b160>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1})
Node-2 :  (1, {'struct': <networkx.classes.graph.Graph object at 0x7f586e931b70>, 'copies': tensor([100.], dtyp

In [4]:
print(rn.rxn_class)
print(rn.mon_rxn_map)
print(rn.dG_map)

{1: [0, 1, 2], 2: [3, 4, 5]}
{('A', 'B'): 0, ('A', 'C'): 1, ('B', 'C'): 2}
{3: [0, 2], 4: [1, 2], 5: [0, 1]}


In [5]:
def decompose_monomers(n1,monomer_set):
        if len(rn.network.in_edges(n1)) == 0:
            return(True,monomer_set)
        else:
            for incoming_edge in rn.network.in_edges(n1):
                flag,monomer_set = decompose_monomers(incoming_edge[0],monomer_set)
                if flag:
                    monomer_set.append(incoming_edge[0])
            return(False,monomer_set)

In [6]:
uid_dict = {}
sys.path.append("../../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    print(n)
    #print(rn.network.nodes()[n])
    
    flag,monomer_set = decompose_monomers(n,[])
    
    print("Node - ", gtostr(rn.network.nodes[n]['struct']), "--", set(monomer_set))
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_dict[(n,k)] = uid

print(uid_dict)
# print(rn.mon_rxns)
# print(gtostr(rn.network.nodes[3]['struct']),rn.network.nodes[3]['struct'].number_of_edges())

# print(rn.network.edges(data=True))

# print(rn.uid_map)

print(rn.rxn_cid)

0
Node -  A -- set()
1
Node -  B -- set()
2
Node -  C -- set()
3
Node -  AB -- {0, 1}
4
Node -  AC -- {0, 2}
5
Node -  BC -- {1, 2}
6
Node -  ABC -- {0, 1, 2}
{(0, 3): 0, (0, 4): 1, (0, 6): 5, (1, 3): 0, (1, 5): 2, (1, 6): 3, (2, 4): 1, (2, 5): 2, (2, 6): 4, (3, 6): 4, (4, 6): 3, (5, 6): 5}
{}


In [7]:
#Changing Initial Conditions
import networkx as nx
#Changin k_on
new_kon = [1.0,1.0,1.0,1.0,1.0,1.0]*np.array([1e0])  #Poor yield Simruntime = 1sec
# new_kon = [ 0.1411,  0.1917,  0.1205, 42.6304, 42.7995, 42.6389]*np.array([1e0])
# new_kon = [0.4052, 0.4145, 0.4049, 5.0401, 5.9947, 5.9940]

update_kon_dict = {}
for edge in rn.network.edges:
    print(rn.network.get_edge_data(edge[0],edge[1]))
    update_kon_dict[edge] = new_kon[uid_dict[edge]]

nx.set_edge_attributes(rn.network,update_kon_dict,'k_on')

# for edge in rn.network.edges:
#     print(rn.network.get_edge_data(edge[0],edge[1]))
# vec_rn = VectorizedRxnNet(rn, dev='cpu',rxn_coupling=True,rx_cid={3:[1],4:[0],5:[2]})
# vec_rn = VectorizedRxnNet(rn, dev='cpu',assoc_is_param=False,dissoc_is_param=True)
vec_rn = VectorizedRxnNet(rn, dev='cpu',assoc_is_param=False,dG_is_param=True,cplx_dG=-60)
print(vec_rn.kon)


{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 0}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 1}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 5}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 0}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 2}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 3}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 1}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 2}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 4}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 4}
{'k_on': 1

## The Equilibrium Solution ##
First we will find the equilibrium solution for this system.

In [8]:
# vec_rn.update_reaction_net(rn,1)
# poly_system = EquilibriumSolver(rn)
# solution = poly_system.solve()
# if solution == None:
#     print("No Equilibrium solution")
# else:
#     print(solution)
#     print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
# print(vec_rn.kon)

In [9]:
# print(vec_rn.rxn_coupling)
# print(rn.rxn_coupling)

In [10]:
# uid_dict = {}
# sys.path.append("../")
# import numpy as np
# from reaction_network import gtostr
# for n in rn.network.nodes():
#     #print(n)
#     #print(rn.network.nodes()[n])
#     for k,v in rn.network[n].items():
#         uid = v['uid']
#         r1 = set(gtostr(rn.network.nodes[n]['struct']))
#         p = set(gtostr(rn.network.nodes[k]['struct']))
#         r2 = p-r1
#         reactants = (r1,r2)
#         uid_val = {'reactants':reactants,'kon':v['k_on'],'score':v['rxn_score'],'koff':v['k_off'],'uid':uid}
#         if uid not in uid_dict.keys():
#             uid_dict[uid] = uid_val
#     print(gtostr(rn.network.nodes[n]['struct']))
#     #for r_set in rn.get_reactant_sets(n):
#     #    print(tuple(r_set))
#     #print(rn.network[n]['struct'])
# ind_sort = np.argsort(vec_rn.kon.detach().numpy())
# for i in ind_sort:
#     print(vec_rn.kon[i])
#     print(uid_dict[i])

In [11]:
# uid_dict = {}
# sys.path.append("../")
# import numpy as np
# from reaction_network import gtostr
# from torch import DoubleTensor as Tensor
# import torch

# node_map = {}
# for node in rn.network.nodes():
#     node_map[gtostr(rn.network.nodes[node]['struct'])] = node

# print(node_map)

# def get_max_edge(n):
#     """
#     Calculates the max rate (k_on) for a given node
#     To find out the maximum flow path to the final complex starting from the current node.
    
#     Can also calculate the total rate of consumption of a node by summing up all rates. 
#     Can tell which component is used quickly.
#     """
#     try:
#         edges = rn.network.out_edges(n)
#         #Loop over all edges
#         #Get attributes
#         if len(edges)==0:
#             return(False)
#         kon_max = -1
#         next_node = -1
        
#         kon_sum = 0
#         for edge in edges:
#             data = rn.network.get_edge_data(edge[0],edge[1])
#             #print(data)
#             #Get uid
#             uid = data['uid']
#             #Get updated kon
#             temp_kon = vec_rn.kon[uid]
#             kon_sum+=temp_kon
            
#             if temp_kon > kon_max:
#                 kon_max = temp_kon
#                 next_node=edge[1]
#         return(kon_max,next_node,kon_sum)
#     except Exception as err:
#         raise(err)
# def get_node_flux(n):
#     total_flux_outedges = 0
#     total_flux_inedges = 0
#     #Go over all the out edges
#     edges_out = rn.network.out_edges(n)
#     if len(edges_out)>0:

#         for edge in edges_out:
#             data = rn.network.get_edge_data(edge[0],edge[1])
#             #print(data)
#             #Get uid
#             uid = data['uid']

#             #Get updated kon
#             temp_kon = vec_rn.kon[uid]

#             #Calculate k_off also
#             std_c = Tensor([1.])
#             l_kon = torch.log(temp_kon)
#             l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
#             koff = torch.exp(l_koff)

#             #Getting conc. of reactants and products
#             #Get product
#             prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
#             #Get other reactant
#             react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

#             #Net flux from this edge = Generation - consumption
#             edge_flux = koff*solution[edge[1]] - temp_kon*(solution[edge[0]])*(solution[node_map[react]])
#             #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

#             print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
#             print("Net flux: ",edge_flux)
#             print("kon : ",temp_kon)
#             print("koff: ",koff)
#             print("Free Energy: ",vec_rn.rxn_score_vec[uid])
#             print("Reaction data OUTWARD: ")
#             print(data)

#             total_flux_outedges+=edge_flux
    
#     #Now go over all the in edges
#     edges_in = rn.network.in_edges(n)
#     react_list = []
#     if len(edges_in) > 0:
#         for edge in edges_in:
#             if edge[0] in react_list:
#                 continue
#             data = rn.network.get_edge_data(edge[0],edge[1])
#             uid = data['uid']


#             #Get generation rates; which would be kon
#             temp_kon = vec_rn.kon[uid]

#             #Get consumption rates; which is k_off
#             std_c = Tensor([1.])
#             l_kon = torch.log(temp_kon)
#             l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
#             koff = torch.exp(l_koff)

#             #Get conc. of reactants and products
#             prod = gtostr(rn.network.nodes[edge[1]]['struct'])
#             #Get other reactant
#             react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
#             react_list.append(node_map[react])
#             #Net flux from this edge = Generation - consumption
#             edge_flux_in = temp_kon*(solution[edge[0]])*(solution[node_map[react]])- koff*solution[edge[1]]
#             #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


#             print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
#             print("Net flux: ",edge_flux_in)
#             print("kon : ",temp_kon)
#             print("koff: ",koff)
#             print("Raction data INWARD: ")
#             print(data)

#             total_flux_inedges+=edge_flux_in
#     net_node_flux = total_flux_outedges + total_flux_inedges
    
#     return(net_node_flux)
# pathway = []
# kon_sumarray = []
# total_con_rate = {}
# net_flux = {}

# if solution != None:
#     for n in rn.network.nodes():

#         n_str = gtostr(rn.network.nodes[n]['struct']) 

#         paths = [n_str]
#         kon_sum = 0
#         temp_node = n
#         max_edge = True
#         consumption_rate = 0
#         if n < len(rn.network.nodes()):#num_monomers:
#     #         print("Current node: ")
#     #         print(n_str)
#             while max_edge:
#                 max_edge = get_max_edge(temp_node)
#                 if max_edge:
#                     total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
#                     temp_node = max_edge[1]
#                     kon_sum += max_edge[0].item()


#     #                 print("Next node: ")
#     #                 print(temp_node)

#                     paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
#                 else:
#                     break
#             pathway.append(paths)
#             kon_sumarray.append(kon_sum)
#             paths=[]

#         print("-------------------------------------------------------------------------------")
#         print("-------------------------------------------------------------------------------")
#         print("|                                                                             |")
#         node_flux = get_node_flux(n)
#         net_flux[gtostr(rn.network.nodes[n]['struct'])] = node_flux
#         print("|                                                                             |")
#         print("-------------------------------------------------------------------------------")
#         print("-------------------------------------------------------------------------------")

#     print(pathway)
#     print(kon_sumarray)
#     #print(total_con_rate)

In [12]:
# if solution != None:
#     for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
#         print(k," : ", v.item())

In [13]:
# if solution !=None:
#     for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
#         print(k," : ", v)



## Using the optimizer with a 1 second simulation runtime ##

In [ ]:
vec_rn.reset(reset_params=True)
optim = Optimizer(reaction_network=vec_rn,
                  sim_runtime=0.1,
                  optim_iterations=5000,
                  learning_rate=1e-2,
                  device='cpu',method="RMSprop",lr_change_step=200)
optim.rn.update_reaction_net(rn)
optim.optimize()

Using CPU
dG of last monomer:  tensor(-20., dtype=torch.float64, grad_fn=<AddBackward0>)
dG of last monomer:  tensor(-20., dtype=torch.float64, grad_fn=<AddBackward0>)
dG of last monomer:  tensor(-20., dtype=torch.float64, grad_fn=<AddBackward0>)
dG of last monomer:  tensor(-20., dtype=torch.float64, grad_fn=<AddBackward0>)
dG of last monomer:  tensor(-20., dtype=torch.float64, grad_fn=<AddBackward0>)
dG of last monomer:  tensor(-20., dtype=torch.float64, grad_fn=<AddBackward0>)
dG of last monomer:  tensor(-20., dtype=torch.float64, grad_fn=<AddBackward0>)
dG of last monomer:  tensor(-20., dtype=torch.float64, grad_fn=<AddBackward0>)
dG of last monomer:  tensor(-20., dtype=torch.float64, grad_fn=<AddBackward0>)
dG of last monomer:  tensor(-20., dtype=torch.float64, grad_fn=<AddBackward0>)
dG of last monomer:  tensor(-20., dtype=torch.float64, grad_fn=<AddBackward0>)
dG of last monomer:  tensor(-20., dtype=torch.float64, grad_fn=<AddBackward0>)
Reaction Parameters before optimization: 


current params: [tensor([0.4109, 0.4109, 0.4109], dtype=torch.float64), tensor([0.0032, 0.0032], dtype=torch.float64)]
Using CPU
dG of last monomer:  tensor(-22.6294, dtype=torch.float64, grad_fn=<AddBackward0>)
Simulation rates:  tensor([4.1088e-01, 4.1088e-01, 4.1095e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        3.1536e-03, 3.1536e-03, 6.1086e-05, 1.9264e-07, 1.9264e-07, 9.9451e-06],
       dtype=torch.float64, grad_fn=<ExpBackward>)
Next time:  tensor(0.1026, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 9 was 76.8%
dG of last monomer:  tensor(-22.6294, dtype=torch.float64, grad_fn=<AddBackward0>)
Current On rates:  tensor([0.4109, 0.4109, 0.4109, 1.0000, 1.0000, 1.0000], dtype=torch.float64,
       grad_fn=<SliceBackward>)
current params: [tensor([0.3632, 0.3632, 0.3633], dtype=torch.float64), tensor([0.0032, 0.0032], dtype=torch.float64)]
Using CPU
dG of last monomer:  tensor(-22.9277, dtype=torc

current params: [tensor([0.1867, 0.1867, 0.1868], dtype=torch.float64), tensor([0.0039, 0.0039], dtype=torch.float64)]
Using CPU
dG of last monomer:  tensor(-24.6169, dtype=torch.float64, grad_fn=<AddBackward0>)
Simulation rates:  tensor([1.8665e-01, 1.8665e-01, 1.8676e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        3.8699e-03, 3.8699e-03, 3.8044e-06, 1.4723e-08, 1.4723e-08, 1.4976e-05],
       dtype=torch.float64, grad_fn=<ExpBackward>)
Next time:  tensor(0.1017, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 20 was 77.0%
dG of last monomer:  tensor(-24.6169, dtype=torch.float64, grad_fn=<AddBackward0>)
Current On rates:  tensor([0.1867, 0.1867, 0.1868, 1.0000, 1.0000, 1.0000], dtype=torch.float64,
       grad_fn=<SliceBackward>)
current params: [tensor([0.1849, 0.1849, 0.1850], dtype=torch.float64), tensor([0.0039, 0.0039], dtype=torch.float64)]
Using CPU
dG of last monomer:  tensor(-24.6622, dtype=tor

current params: [tensor([0.1687, 0.1687, 0.1688], dtype=torch.float64), tensor([0.0043, 0.0043], dtype=torch.float64)]
Using CPU
dG of last monomer:  tensor(-25.0488, dtype=torch.float64, grad_fn=<AddBackward0>)
Simulation rates:  tensor([1.6865e-01, 1.6865e-01, 1.6878e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        4.3394e-03, 4.3394e-03, 2.2325e-06, 9.6875e-09, 9.6875e-09, 1.8830e-05],
       dtype=torch.float64, grad_fn=<ExpBackward>)
Next time:  tensor(0.1003, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 30 was 75.9%
dG of last monomer:  tensor(-25.0488, dtype=torch.float64, grad_fn=<AddBackward0>)
Current On rates:  tensor([0.1687, 0.1687, 0.1688, 1.0000, 1.0000, 1.0000], dtype=torch.float64,
       grad_fn=<SliceBackward>)
current params: [tensor([0.1667, 0.1667, 0.1669], dtype=torch.float64), tensor([0.0044, 0.0044], dtype=torch.float64)]
Using CPU
dG of last monomer:  tensor(-25.0909, dtype=tor

Next time:  tensor(0.1017, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 40 was 75.0%
dG of last monomer:  tensor(-25.4735, dtype=torch.float64, grad_fn=<AddBackward0>)
Current On rates:  tensor([0.1488, 0.1488, 0.1490, 1.0000, 1.0000, 1.0000], dtype=torch.float64,
       grad_fn=<SliceBackward>)
current params: [tensor([0.1468, 0.1468, 0.1469], dtype=torch.float64), tensor([0.0048, 0.0048], dtype=torch.float64)]
Using CPU
dG of last monomer:  tensor(-25.5173, dtype=torch.float64, grad_fn=<AddBackward0>)
Simulation rates:  tensor([1.4675e-01, 1.4675e-01, 1.4692e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        4.7727e-03, 4.7727e-03, 1.2164e-06, 5.8055e-09, 5.8055e-09, 2.2778e-05],
       dtype=torch.float64, grad_fn=<ExpBackward>)
Next time:  tensor(0.1029, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 41 was 75.1%
dG of l

...

Next time:  tensor(0.1005, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 809 was 69.3%
dG of last monomer:  tensor(-29.3323, dtype=torch.float64, grad_fn=<AddBackward0>)
Current On rates:  tensor([0.0987, 0.0987, 0.1010, 1.0000, 1.0000, 1.0000], dtype=torch.float64,
       grad_fn=<SliceBackward>)
current params: [tensor([0.1594, 0.1594, 0.0994], dtype=torch.float64), tensor([0.0216, 0.0216], dtype=torch.float64)]
Using CPU
dG of last monomer:  tensor(-28.3749, dtype=torch.float64, grad_fn=<AddBackward0>)
Simulation rates:  tensor([1.5943e-01, 1.5943e-01, 9.9408e-02, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        2.1640e-02, 2.1640e-02, 4.7244e-08, 1.0224e-09, 1.0224e-09, 4.6830e-04],
       dtype=torch.float64, grad_fn=<ExpBackward>)
Next time:  tensor(0.1019, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 810 was 74.3%
dG of

current params: [tensor([0.1480, 0.1480, 0.1499], dtype=torch.float64), tensor([0.0218, 0.0218], dtype=torch.float64)]
Using CPU
dG of last monomer:  tensor(-28.5426, dtype=torch.float64, grad_fn=<AddBackward0>)
Simulation rates:  tensor([1.4800e-01, 1.4800e-01, 1.4994e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        2.1846e-02, 2.1846e-02, 6.0258e-08, 1.3164e-09, 1.3164e-09, 4.7726e-04],
       dtype=torch.float64, grad_fn=<ExpBackward>)
Next time:  tensor(0.1017, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 820 was 75.0%
dG of last monomer:  tensor(-28.5426, dtype=torch.float64, grad_fn=<AddBackward0>)
Current On rates:  tensor([0.1480, 0.1480, 0.1499, 1.0000, 1.0000, 1.0000], dtype=torch.float64,
       grad_fn=<SliceBackward>)
current params: [tensor([0.1468, 0.1468, 0.1488], dtype=torch.float64), tensor([0.0219, 0.0219], dtype=torch.float64)]
Using CPU
dG of last monomer:  tensor(-28.5607, dtype=to

current params: [tensor([0.1356, 0.1356, 0.1377], dtype=torch.float64), tensor([0.0221, 0.0221], dtype=torch.float64)]
Using CPU
dG of last monomer:  tensor(-28.7364, dtype=torch.float64, grad_fn=<AddBackward0>)
Simulation rates:  tensor([1.3560e-01, 1.3560e-01, 1.3774e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        2.2052e-02, 2.2052e-02, 4.5605e-08, 1.0057e-09, 1.0057e-09, 4.8631e-04],
       dtype=torch.float64, grad_fn=<ExpBackward>)
Next time:  tensor(0.1006, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 830 was 73.8%
dG of last monomer:  tensor(-28.7364, dtype=torch.float64, grad_fn=<AddBackward0>)
Current On rates:  tensor([0.1356, 0.1356, 0.1377, 1.0000, 1.0000, 1.0000], dtype=torch.float64,
       grad_fn=<SliceBackward>)
current params: [tensor([0.1343, 0.1343, 0.1365], dtype=torch.float64), tensor([0.0221, 0.0221], dtype=torch.float64)]
Using CPU
dG of last monomer:  tensor(-28.7576, dtype=to

Next time:  tensor(0.1016, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 840 was 72.7%
dG of last monomer:  tensor(-28.9645, dtype=torch.float64, grad_fn=<AddBackward0>)
Current On rates:  tensor([0.1221, 0.1221, 0.1245, 1.0000, 1.0000, 1.0000], dtype=torch.float64,
       grad_fn=<SliceBackward>)
current params: [tensor([0.1207, 0.1207, 0.1231], dtype=torch.float64), tensor([0.0223, 0.0223], dtype=torch.float64)]
Using CPU
dG of last monomer:  tensor(-28.9896, dtype=torch.float64, grad_fn=<AddBackward0>)
Simulation rates:  tensor([1.2070e-01, 1.2070e-01, 1.2311e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        2.2279e-02, 2.2279e-02, 3.1642e-08, 7.0495e-10, 7.0495e-10, 4.9636e-04],
       dtype=torch.float64, grad_fn=<ExpBackward>)
Next time:  tensor(0.1026, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 841 was 72.8%
dG of

current params: [tensor([0.1058, 0.1058, 0.1085], dtype=torch.float64), tensor([0.0225, 0.0225], dtype=torch.float64)]
Using CPU
dG of last monomer:  tensor(-29.2712, dtype=torch.float64, grad_fn=<AddBackward0>)
Simulation rates:  tensor([1.0582e-01, 1.0582e-01, 1.0850e-01, 1.0000e+00, 1.0000e+00, 1.0000e+00,
        2.2485e-02, 2.2485e-02, 2.1044e-08, 4.7317e-10, 4.7317e-10, 5.0559e-04],
       dtype=torch.float64, grad_fn=<ExpBackward>)
Next time:  tensor(0.1014, dtype=torch.float64, grad_fn=<AddBackward0>)
Next time larger than simulation runtime. Ending simulation.
yield on sim iteration 851 was 70.7%
dG of last monomer:  tensor(-29.2712, dtype=torch.float64, grad_fn=<AddBackward0>)
Current On rates:  tensor([0.1058, 0.1058, 0.1085, 1.0000, 1.0000, 1.0000], dtype=torch.float64,
       grad_fn=<SliceBackward>)
current params: [tensor([0.1043, 0.1043, 0.1070], dtype=torch.float64), tensor([0.0225, 0.0225], dtype=torch.float64)]
Using CPU
dG of last monomer:  tensor(-29.3028, dtype=to

In [ ]:
keq = 0.1178/0.0676
g = -np.log(keq*1e6)
print(g)

In [ ]:
print(optim.yield_per_iter[-1])

In [ ]:
def calc_var(v1,v2):
    sq_sum=0
    for i in range(len(v1)):
        sq_sum=(v1[i]-v2[i])**2+sq_sum
    
    sq_sum = ((sq_sum)**0.5)/(len(v1)-1)
    return(sq_sum)

In [ ]:
yields= []
final_params=[]
for i in range(len(optim.final_yields)):
    yields.append(optim.final_yields[i].item())
#     print(optim.final_solns[i].numpy())
    final_params.append(optim.final_solns[i].numpy())

sort_indx=np.argsort(np.array(yields))
sorted_yields=np.array(yields)[sort_indx]
sorted_params = np.array(final_params)[sort_indx]

p0 = sorted_params[0]
var_params = []
for i in range(len(sorted_params)):
    var_params.append(calc_var(p0,sorted_params[i]))
    
arg_indx = np.argsort(np.array(var_params))
sorted_var = np.array(var_params)[arg_indx]

print(sorted_var[0])
print(sorted_var[-1])
print("Yield: ",sorted_yields[arg_indx[0]],"\nParams: ",sorted_params[arg_indx[0]])

print("Yield: ",sorted_yields[arg_indx[-1]],"\nParams: ",sorted_params[arg_indx[-1]])
print("Max Yield: ",sorted_yields[-1],"\nParams: ",sorted_params[-1])

In [ ]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_val = {'reactants':reactants,'kon':v['k_on'],'score':v['rxn_score'],'koff':v['k_off'],'uid':uid}
        if uid not in uid_dict.keys():
            uid_dict[uid] = uid_val
    print(gtostr(rn.network.nodes[n]['struct']))
    #for r_set in rn.get_reactant_sets(n):
    #    print(tuple(r_set))
    #print(rn.network[n]['struct'])
ind_sort = np.argsort(vec_rn.kon.detach().numpy())
for i in ind_sort:
    print(vec_rn.kon[i])
    print(uid_dict[i])

In [ ]:
uid_dict = {}
sys.path.append("../")
import numpy as np
from reaction_network import gtostr
from torch import DoubleTensor as Tensor
import torch

node_map = {}
for node in rn.network.nodes():
    node_map[gtostr(rn.network.nodes[node]['struct'])] = node

print(node_map)

def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        kon_max = -1
        next_node = -1

        kon_sum = 0
        total_flux_outedges = 0
        total_flux_inedges = 0
        if len(edges)==0:
            return(False)
            
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
             
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

        
def get_node_flux(n):
    total_flux_outedges = 0
    total_flux_inedges = 0
    #Go over all the out edges
    edges_out = rn.network.out_edges(n)
    if len(edges_out)>0:

        for edge in edges_out:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]

            #Calculate k_off also
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Getting conc. of reactants and products
            #Get product
            prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

            #Net flux from this edge = Generation - consumption
            edge_flux = koff*vec_rn.copies_vec[edge[1]] - temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])
            #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

            print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
            print("Net flux: ",edge_flux)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Reaction data OUTWARD: ")
            print(data)

            total_flux_outedges+=edge_flux
    
    #Now go over all the in edges
    edges_in = rn.network.in_edges(n)
    react_list = []
    if len(edges_in) > 0:
        for edge in edges_in:
            if edge[0] in react_list:
                continue
            data = rn.network.get_edge_data(edge[0],edge[1])
            uid = data['uid']


            #Get generation rates; which would be kon
            temp_kon = vec_rn.kon[uid]

            #Get consumption rates; which is k_off
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Get conc. of reactants and products
            prod = gtostr(rn.network.nodes[edge[1]]['struct'])
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
            react_list.append(node_map[react])
            #Net flux from this edge = Generation - consumption
            edge_flux_in = temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])- koff*vec_rn.copies_vec[edge[1]]
            #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


            print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
            print("Net flux: ",edge_flux_in)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Raction data INWARD: ")
            print(data)

            total_flux_inedges+=edge_flux_in
    net_node_flux = total_flux_outedges + total_flux_inedges
    
    return(net_node_flux)
    
pathway = []
kon_sumarray = []
total_con_rate = {}
net_flux = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")
    print("|                                                                             |")
    node_flux = get_node_flux(n)
    net_flux[gtostr(rn.network.nodes[n]['struct'])] = node_flux
    print("|                                                                             |")
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")

print(pathway)
print(kon_sumarray)

#print(total_con_rate)

In [ ]:
for k,v in sorted(total_con_rate.items(),key=lambda x : x[1]):
    print(k," : ", v.item())

In [ ]:
for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
    print(k," : ", v)

print(vec_rn.copies_vec)

In [ ]:
for n in rn.network.nodes():
    print(gtostr(rn.network.nodes[n]['struct']), " : ", n)

Let's first visualize some of the data.

**Without any optimization**


In [ ]:
nodes_list = ['A','B','C','ABC','AB','BC','AC']
#nodes_list = ['AB','BMS','ABS','AMS','ABMS','AM','BM','MS']
optim.plot_observable(0,nodes_list)


**After 750 optimization iterations**


In [ ]:

optim.plot_observable(-1,nodes_list)


In [ ]:
optim.plot_yield()

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots()
ax.plot(optim.yield_per_iter)
f_dict = {'fontsize':22}
ax.set_ylabel("Yield (%)",fontdict=f_dict)
ax.set_xlabel("Iterations",fontdict=f_dict)
plt.tick_params(axis='both',labelsize=14.0)
ax.hlines(y=0.999,xmin=0,xmax=5000,linewidth=2.5,linestyle='dashed',label='Eq. yield')
ax.legend(fontsize=16)

It seems like we've found a stable solution that produces greater yield than equilibrium. This should be thermodynamically
impossible. Let's try to find an explanation. We'll run simulations using the learned optimal parameters at a few different
timescales.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(1, 4)
optim_rn = optim.rn
for i, runtime in enumerate([10, 100, 500,1000]):
    optim_rn.reset()
    sim = VecSim(optim_rn, runtime, device='cpu')
    y = sim.simulate()
    sim.plot_observable(nodes_list,ax=ax[i])
    ax[i].set_title("runtime: " + str(runtime) + " seconds")
fig.set_size_inches(18, 6)
plt.show()

In [ ]:
def get_max_edge(n):
    """
    Calculates the max rate (k_on) for a given node
    To find out the maximum flow path to the final complex starting from the current node.
    
    Can also calculate the total rate of consumption of a node by summing up all rates. 
    Can tell which component is used quickly.
    """
    try:
        edges = rn.network.out_edges(n)
        #Loop over all edges
        #Get attributes
        kon_max = -1
        next_node = -1

        kon_sum = 0
        total_flux_outedges = 0
        total_flux_inedges = 0
        if len(edges)==0:
            return(False)
            
        for edge in edges:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]
            kon_sum+=temp_kon
            
            if temp_kon > kon_max:
                kon_max = temp_kon
                next_node=edge[1]
             
        return(kon_max,next_node,kon_sum)
    except Exception as err:
        raise(err)

        
def get_node_flux(n):
    total_flux_outedges = 0
    total_flux_inedges = 0
    #Go over all the out edges
    edges_out = rn.network.out_edges(n)
    if len(edges_out)>0:

        for edge in edges_out:
            data = rn.network.get_edge_data(edge[0],edge[1])
            #print(data)
            #Get uid
            uid = data['uid']

            #Get updated kon
            temp_kon = vec_rn.kon[uid]

            #Calculate k_off also
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Getting conc. of reactants and products
            #Get product
            prod = gtostr(rn.network.nodes[edge[1]]['struct']) 
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))

            #Net flux from this edge = Generation - consumption
            edge_flux = koff*vec_rn.copies_vec[edge[1]] - temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])
            #edge_flux = koff*vec_rn.copies_vec[edge[1]] 

            print("Reaction: ", gtostr(rn.network.nodes[edge[0]]['struct']), "+",react," -> ",prod)
            print("Net flux: ",edge_flux)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Reaction data OUTWARD: ")
            print(data)

            total_flux_outedges+=edge_flux
    
    #Now go over all the in edges
    edges_in = rn.network.in_edges(n)
    react_list = []
    if len(edges_in) > 0:
        for edge in edges_in:
            if edge[0] in react_list:
                continue
            data = rn.network.get_edge_data(edge[0],edge[1])
            uid = data['uid']


            #Get generation rates; which would be kon
            temp_kon = vec_rn.kon[uid]

            #Get consumption rates; which is k_off
            std_c = Tensor([1.])
            l_kon = torch.log(temp_kon)
            l_koff = (vec_rn.rxn_score_vec[uid] * 1. / (vec_rn._R * vec_rn._T)) + l_kon + torch.log(std_c)
            koff = torch.exp(l_koff)

            #Get conc. of reactants and products
            prod = gtostr(rn.network.nodes[edge[1]]['struct'])
            #Get other reactant
            react = "".join(sorted(list(set(prod) - set(gtostr(rn.network.nodes[edge[0]]['struct']) ))))
            react_list.append(node_map[react])
            #Net flux from this edge = Generation - consumption
            edge_flux_in = temp_kon*(vec_rn.copies_vec[edge[0]])*(vec_rn.copies_vec[node_map[react]])- koff*vec_rn.copies_vec[edge[1]]
            #edge_flux_in = koff*vec_rn.copies_vec[edge[1]]
            


            print("Reaction: ", prod ," -> ",gtostr(rn.network.nodes[edge[0]]['struct']), "+",react)
            print("Net flux: ",edge_flux_in)
            print("kon : ",temp_kon)
            print("koff: ",koff)
            print("Raction data INWARD: ")
            print(data)

            total_flux_inedges+=edge_flux_in
    net_node_flux = total_flux_outedges + total_flux_inedges
    
    return(net_node_flux)
    
pathway = []
kon_sumarray = []
total_con_rate = {}
net_flux = {}
for n in rn.network.nodes():
    
    n_str = gtostr(rn.network.nodes[n]['struct']) 
    
    paths = [n_str]
    kon_sum = 0
    temp_node = n
    max_edge = True
    consumption_rate = 0
    if n < len(rn.network.nodes()):#num_monomers:
#         print("Current node: ")
#         print(n_str)
        while max_edge:
            max_edge = get_max_edge(temp_node)
            if max_edge:
                total_con_rate[gtostr(rn.network.nodes[temp_node]['struct'])] = max_edge[2]
                
                temp_node = max_edge[1]
                kon_sum += max_edge[0].item()
                
                
#                 print("Next node: ")
#                 print(temp_node)

                paths.append(gtostr(rn.network.nodes[temp_node]['struct']))
            else:
                break
        pathway.append(paths)
        kon_sumarray.append(kon_sum)
        paths=[]
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")
    print("|                                                                             |")
    node_flux = get_node_flux(n)
    net_flux[gtostr(rn.network.nodes[n]['struct'])] = node_flux
    print("|                                                                             |")
    print("-------------------------------------------------------------------------------")
    print("-------------------------------------------------------------------------------")

print(pathway)
print(kon_sumarray)

#print(total_con_rate)

In [ ]:
for k,v in sorted(net_flux.items(),key=lambda x : x[1]):
    print(k," : ", v)

print(vec_rn.copies_vec)
print(vec_rn.kon)

In [ ]:
print(solution)
poly_system = EquilibriumSolver(rn)
solution = poly_system.solve(init_val=vec_rn.copies_vec.detach().numpy().tolist())
#solution = poly_system.solve(verifyBool = False)
if solution == None:
    print("No Equilibrium solution")
else:
    print(solution)
    print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
print(vec_rn.kon)

In [ ]:
print(solution)
poly_system = EquilibriumSolver(rn)
solution = poly_system.solve(verifyBool = False)
if solution == None:
    print("No Equilibrium solution")
else:
    print(solution)
    print("Equilibrium expected yield: ", 100 * solution[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')
print(vec_rn.kon)

In [ ]:
def corr_matrix(M):
    return(np.corrcoef(M))

#Select data range to calculate coorelations
yield_thresh_min = 0.8
yield_thresh_max = 0.9
mask1 = (np.array(optim.final_yields) >= yield_thresh_min) & (np.array(optim.final_yields) < yield_thresh_max)
print(mask1)

coor_params = np.zeros((6,1))
for i in range(len(mask1)):
    if mask1[i]:
        coor_params = np.concatenate((coor_params,np.reshape(optim.final_solns[i].numpy(),(6,1))),axis=1)

corr_matrix = corr_matrix(coor_params)

# %matplotlib notebook
fig_c,ax_c = plt.subplots()
hm = ax_c.imshow(corr_matrix)
ax_labels=['k1','k2','k3','k4','k5','k6']
ax_c.set_xticks(np.arange(len(ax_labels)))
ax_c.set_yticks(np.arange(len(ax_labels)))
ax_c.set_xticklabels(ax_labels)
ax_c.set_yticklabels(ax_labels)
plt.setp(ax_c.get_xticklabels(),rotation=45, ha='center',fontsize=10,va='top')
plt.setp(ax_c.get_yticklabels(),va='center',fontsize=10)
fig_c.colorbar(hm,ax=ax_c,aspect=40,label="Corr Coeff",orientation='horizontal',panchor=(0.5,0.0),pad=0.2,shrink=0.5)
# ax_c.grid()

plt.show()
label = "Number of samples: %.2f %s" %(coor_params.shape[1]*100/len(optim.final_solns),"%")
yield_label = "%.1f - %.1f Yield range\n" %(yield_thresh_min,yield_thresh_max)
ax_c.set_title(yield_label+label)

Clearly, the equilibrium reached by the system still matches the equilibrium solution. We have however found a set of parameters that can increase available complete AP2 at some point before equilibrium to levels significantly higher than at equilibrium. We don't observe any trapping, but have uncovered an interesting effect. 

Now we'll move on to looking at ARP23. This is 7 subunits, which drastically increases the number of possible reactions. Expect longer runtimes. 